# Inference with the model on openEO

This notebook demonstrates how to use a trained model for inference with onnxruntime in an udf.

In [1]:
# Make the connection and authenticate
import openeo

#Choose preferred backend
#conn = openeo.connect("https://openeo.vito.be/openeo/").authenticate_oidc()
conn = openeo.connect("https://openeo.dataspace.copernicus.eu/").authenticate_oidc()


Authenticated using refresh token.


A workflow for processing Sentinel-2 Level-2A earth observation data using openEO in orden to obtain the desired data, compatible with the classifier


In [2]:
# Import necessary utility function
from utils import  timesteps_as_bands

# Load Sentinel-2 Level-2A data cube
s2_cube = conn.load_collection(
                collection_id = "SENTINEL2_L2A",
                spatial_extent = 
                {"west": 12.17,
                  "east": 12.18,
                  "south": 51.46,
                  "north": 51.47},
                temporal_extent = ["2022-01-01","2022-12-31"],
                bands = ["B02","B03","B04","B05","B06","B07","B08","B8A","B11","B12","SCL"],
                properties = {"eo:cloud_cover": lambda x: x.lte(65)}
)


# Create weekly composites by taking the mean
s2_cube = s2_cube.process("mask_scl_dilation", data=s2_cube, scl_band_name="SCL").filter_bands(s2_cube.metadata.band_names[:-1])

# Create weekly composites by taking the mean
s2_cube = s2_cube.aggregate_temporal_period(
    period = "week",
    reducer = "mean"
)

# Fill gaps in the data using linear interpolation
s2_cube = s2_cube.apply_dimension(
    dimension = "t",
    process = "array_interpolate_linear"
)

# Filter out January and December data to ensure 43 weeks of data
s2_cube = s2_cube.filter_temporal(["2022-02-01","2022-11-30"]) # This should result in 43 weeks

# Rearrange cube from (time, x, y bands) to (x, y, time*bands)
s2_cube = timesteps_as_bands(s2_cube, 43)


Load in the model from the vito artifactory and the required dependencies and excecute the prediction job. The output is stored as a netCDF in the local directory

In [3]:

# Supply the model as a URL. The model is stored in artifactory
model_url = "https://artifactory.vgt.vito.be/artifactory/auxdata-public/photovoltaic/random_forest.onnx"
udf = openeo.UDF.from_file(
    "udf_rf_onnx.py", 
    context={
        "model_url": model_url
    }
)

# Add the onnx dependencies to the job options. You can reuse this existing dependencies archive
dependencies_url = "https://artifactory.vgt.vito.be/artifactory/auxdata-public/openeo/onnx_dependencies_1.16.3.zip"
job_options = {
    "udf-dependency-archives": [
        f"{dependencies_url}#onnx_deps"
    ],
}


# Reduce the bands dimesnion to a single prediction using the udf
prediction = s2_cube.reduce_bands(reducer=udf)

prediction_job = prediction.execute_batch(
    "./photovoltaic_prediction.nc",
    job_options=job_options,
    title="photovoltaic_prediction"
)


0:00:00 Job 'j-24040344e1094fcebe12423d94038077': send 'start'
0:00:17 Job 'j-24040344e1094fcebe12423d94038077': created (progress N/A)
0:00:22 Job 'j-24040344e1094fcebe12423d94038077': created (progress N/A)
0:00:29 Job 'j-24040344e1094fcebe12423d94038077': running (progress N/A)
0:00:39 Job 'j-24040344e1094fcebe12423d94038077': running (progress N/A)
0:00:49 Job 'j-24040344e1094fcebe12423d94038077': running (progress N/A)
0:01:02 Job 'j-24040344e1094fcebe12423d94038077': running (progress N/A)
0:01:17 Job 'j-24040344e1094fcebe12423d94038077': running (progress N/A)
0:01:37 Job 'j-24040344e1094fcebe12423d94038077': running (progress N/A)
0:02:01 Job 'j-24040344e1094fcebe12423d94038077': running (progress N/A)
0:02:31 Job 'j-24040344e1094fcebe12423d94038077': running (progress N/A)
0:03:08 Job 'j-24040344e1094fcebe12423d94038077': running (progress N/A)
0:03:55 Job 'j-24040344e1094fcebe12423d94038077': running (progress N/A)
0:04:54 Job 'j-24040344e1094fcebe12423d94038077': running (pr

JobFailedException: Batch job 'j-24040344e1094fcebe12423d94038077' didn't finish successfully. Status: error (after 0:07:08).